# Data collection

As my goal is to learn data science, I want to use only data provided directly by Riot Games, the developers of League of Legends. I will utilize their API to extract data, ensuring that the data is as raw and unprocessed as possible.

Notes: 
-   Riot's API key for amateur developers is only valid for 24 hours after generation. Please remember to refresh your key regularly and store it in the `.env` file to ensure the project's code functions correctly.
-   The limit rate for requests is 20/seconds and 50/minutes

## First API call and size estimate

Since the request rate is limited by frequency and size, a good starting point for this project's data collection would be to estimate the volume of data needed. Refer to the [Scope Notebook](scope.ipynb) for more details about the project's scope.

It will also be a good time to get familiar with the API.

In [ ]:
%pip install requests -r requirements.txt
from requests import request


import json
import os

API_KEY = os.getenv("RIOT_API_KEY")
QUEUE = "RANKED_SOLO_5x5"
REGION = "euw1"
TIER = "EMERALD"                    # BRONZE, SILVER, GOLD, PLATINUM, DIAMOND, EMERALD, MASTER, GRANDMASTER, CHALLENGER
DIVISION = "I"                      # I, II, III, IV
PAGE = "1"


response = request(
    method="GET",
    url=f"https://{REGION}.api.riotgames.com/lol/league/v4/entries/{QUEUE}/{TIER}/{DIVISION}?page=1&api_key={API_KEY}",
    headers={
        "Accept": "application/json",
        "Content-Type": "application/json",
    },
) 

print(response.status_code)
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))
else:
    print(f"Error: {response.status_code}")
    print(response.text)
    # Handle the error as needed

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\zaeld\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'requests'